In [1]:
#Performing sentiment analysis on Twitter data involves five steps:

#Gather relevant Twitter data.

#Clean your data using pre-processing techniques.

#Create a sentiment analysis machine learning model.

#Analyze your Twitter data using your sentiment analysis model.

#Visualize the results of your Twitter sentiment analysis.

In [2]:
#Sentiment Analysis with Twitter API

#Used the twitter_api to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

#Used descriptive statistics to answer the following questions:

#Which coin had the highest mean positive score?

#Which coin had the highest negative score?

#Which coin had the highest positive score?

In [ ]:
#

In [3]:
!pip install textblob

In [4]:
pip install twitter api

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Initial imports
import os
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

from textblob import TextBlob

from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/tylerscott/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
# Import Libraries

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
import tweepy

# Load env variables and read the News API Key enviroment variable
load_dotenv()
Twitter_api_key = os.getenv("TWITTER_API_KEY")


In [8]:
#Get Twitter API credentials

Twitter_api_key = "zYWhAgVMgnUbXk7oRUbha7CaX"

Twitter_api_secrets = "sK7fm012boPiMLUem3niYb6H6xpgFktMdQjYNT6PkzWtjoXJBH"

Twitter_access_token = "924814822956503040-vLpCSjRaJFeSxfmKYpySEB6v9jbhWcX"

Twitter_access_secret_token = "IVoy7yau6YM51hLMzD5EAyDegIC27yCDjF2g9kvitBdvU"

In [9]:
auth = tweepy.OAuthHandler(Twitter_api_key, Twitter_api_secrets)
auth.set_access_token(Twitter_access_token, Twitter_access_secret_token)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [10]:
# Load the data Bitcoin_tweets.csv (Sourced by way of KAGGLE)
bitcoin_df = pd.read_csv('Bitcoin_tweets.csv')

# Preview the data
bitcoin_df.head(5)

/Users/tylerscott/opt/anaconda3/envs/nlpenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4,5,6,7,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False


In [11]:
 # VADER Sentiment -Get descriptive stats from the Bitcoin_tweets DF.
bitcoin_df.describe()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
count,3543820,1789601,3111337,3543760,3543718.0,3543718.0,3543718.0,3543718,3543718,3543718,3526657,3540087,3543400
unique,523059,82508,530750,503013,77627.0,30473.0,133408.0,21,2755092,3458875,793447,2451,1
top,ExchangeWhales🐋,United States,"Market analysis & recaps, scalping leverage ca...",2021-10-17 23:36:14,0.0,0.0,0.0,False,2022-05-31 06:02:30,💸 Earn free #BTC and multiply crypto up to 15%...,['Bitcoin'],Twitter for Android,False
freq,16788,42396,21470,16788,44676.0,84250.0,114481.0,2607878,373,1351,448121,1062610,3543400


In [12]:
#Vader Sentiment Score for Bitcoin

# Sentiment calculation based on compound score
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    return result

In [13]:
#

In [14]:
# # Sentiment scores dictionaries

text_sent = {
    "text_compound": [],
    "text_pos": [],
    "text_neu": [],
    "text_neg": [],
    "text_sent": [],
}

In [15]:
# Get sentiment for the text 
for index, row in bitcoin_df.iterrows():
    try:
        # Sentiment scoring with VADER
        text_sentiment = analyzer.polarity_scores(row["text"])
        text_sent["text_compound"].append(text_sentiment["compound"])
        text_sent["text_pos"].append(text_sentiment["pos"])
        text_sent["text_neu"].append(text_sentiment["neu"])
        text_sent["text_neg"].append(text_sentiment["neg"])
        text_sent["text_sent"].append(get_sentiment(text_sentiment["compound"]))
    except AttributeError:
        pass

In [16]:
# Attaching sentiment columns to the New DataFrame
text_sentiment_df = pd.DataFrame(text_sent)
bitcoin_df = bitcoin_df.join(text_sentiment)

bitcoin_df.head(5)

AttributeError: 'builtin_function_or_method' object has no attribute 'is_unique'

In [ ]:
#Analyzing Sentiments Results


In [ ]:
#

In [17]:
# Load the data Ethereum_tweets.csv (Sourced by way of KAGGLE)
ethereum_df = pd.read_csv('Ethereum_tweets.csv')

# Preview the data
ethereum_df.head(5)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,#1 Crypto Currency TRADING,NaN,"Profit thought trading, Learn , Trade & Earn💲....",2021-11-10 10:42:01+00:00,4,0,0,False,2021-11-15 07:26:39+00:00,Get upto 1000 that’s 10x with our premium spot...,NaN,Twitter for Android,False
1,Reyrey,Kuala Lumpur,Artist | NFT Broker & Collector for AnideaNFT ...,2011-03-07 07:22:57+00:00,343,650,589,False,2021-11-15 07:26:35+00:00,"Adopted a Hypocat #053, “Crazy Rich Cat” 🤑💵💵💵🤩...","['SupportingCreators', 'CryptoArt', 'CryptoArt...",Twitter for iPhone,False
2,akter jahan,NaN,NaN,2021-10-28 13:28:05+00:00,1,2,0,False,2021-11-15 07:26:32+00:00,Top 10 Coins by Social Engagement over the las...,"['LunarCrush', 'bitcoin', 'dogecoin']",Twitter Web App,False
3,riyan,NaN,i always paricipated your project,2021-09-16 16:59:11+00:00,11,127,99,False,2021-11-15 07:26:30+00:00,Requesting faucet funds into 0x5278942b39deD8c...,"['Rinkeby', 'Ethereum']",Twitter Web App,False
4,Michelle Eriksen,NaN,"Art Creator, NFT, Crypto",2021-08-11 06:40:08+00:00,522,1428,16,False,2021-11-15 07:26:30+00:00,New Listing BTS 0.008ETH !! \n#NFTs #nftcolle...,"['NFTs', 'nftcollector', 'NFTCommunity', 'art'...",Twitter Web App,False


In [18]:
 # VADER Sentiment - Get descriptive stats from the Ethereum_tweets DF.
ethereum_df.describe()

,user_followers,user_friends,user_favourites
count,1.000000e+04,10000.000000,10000.00000
mean,3.407436e+03,879.184100,3594.31600
std,4.144713e+04,3700.326665,15861.80172
min,0.000000e+00,0.000000,0.00000
25%,2.600000e+01,36.000000,32.00000
50%,1.530000e+02,178.000000,336.50000
75%,7.720000e+02,667.250000,1948.25000
max,3.206537e+06,200574.000000,951778.00000


In [19]:
#Vader Sentiment Score for Ethereum

# Sentiment calculation based on compound score
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    return result

In [20]:
# # Sentiment scores dictionaries

text_sent = {
    "text_compound": [],
    "text_pos": [],
    "text_neu": [],
    "text_neg": [],
    "text_sent": [],
}

In [21]:
# Get sentiment for the eth_text 
for index, row in ethereum_df.iterrows():
    try:
        # Sentiment scoring with VADER
        text_sentiment = analyzer.polarity_scores(row["text"])
        text_sent["text_compound"].append(text_sentiment["compound"])
        text_sent["text_pos"].append(text_sentiment["pos"])
        text_sent["text_neu"].append(text_sentiment["neu"])
        text_sent["text_neg"].append(text_sentiment["neg"])
        text_sent["text_sent"].append(get_sentiment(text_sentiment["compound"]))
    except AttributeError:
        pass

In [22]:
# Attaching sentiment columns to the News DataFrame
# Attaching sentiment columns to the New DataFrame
text_sentiment_df = pd.DataFrame(text_sent)
ethereum_df = ethereum_df.join(text_sentiment)

bitcoin_df.head(5)

AttributeError: 'builtin_function_or_method' object has no attribute 'is_unique'

In [ ]:
#

In [ ]:
#

In [23]:
# Load the data Dodgecoin-tweets.csv (Sourced by way of KAGGLE)
dogecoin_df = pd.read_csv('Tweets-Dogecoin.csv')

# Preview the data
dogecoin_df.head(5)

,date,time,timezone,username,name,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,video,thumbnail,reply_to
0,20-04-2021,17:57:09,530,sunnyepposo,congrats ✨,I invested $16k a couple of weeks ago when a l...,en,None,['http://discord.gg/5zwNm6Kn9v'],['https://pbs.twimg.com/media/EzaspZBVEAMHmZE....,0,0,0,"['dogeday420', 'dogeday', 'doge420', 'dogecoin']",['doge'],1,https://pbs.twimg.com/media/EzaspZBVEAMHmZE.jpg,None
1,20-04-2021,17:57:07,530,pickndng,andrea || busy sa exams at pts kuno🍀✨,Bought in for $16k. My #dogecoin will make me ...,en,None,['http://discord.gg/5zwNm6Kn9v'],['https://pbs.twimg.com/media/EzaspdbVgAgSD_D....,0,0,0,"['dogecoin', 'doge', 'dogeday']",None,1,https://pbs.twimg.com/media/EzaspdbVgAgSD_D.jpg,None
2,20-04-2021,17:57:07,530,s4y4n9_semu4,#SayangiSemesta,I invested $16k a couple of weeks ago when a l...,en,None,['http://discord.gg/5zwNm6Kn9v'],['https://pbs.twimg.com/media/EzaspfMVoAAmNIi....,0,0,0,"['dogeday420', 'dogeday', 'doge420', 'dogecoin']",['doge'],1,https://pbs.twimg.com/media/EzaspfMVoAAmNIi.jpg,None
3,20-04-2021,17:57:06,530,dogecoinsky,dogecoinsky,"looking back in 2010/11, BTC had low value, hi...",en,None,None,None,0,0,0,"['dogecoin', 'dogeday', 'dogecoinsky']",None,0,NaN,None
4,20-04-2021,17:57:04,530,dogeman0401,jay,Looks like the hype of today isn’t coming to f...,en,None,None,None,0,0,0,"['doge420', 'dogeday420', 'doge', 'dogecoin']",None,0,NaN,None


In [24]:
 # VADER Sentiment - Get descriptive stats from the dodgecoin_tweets DF.
dogecoin_df.describe()

,timezone,replies_count,retweets_count,likes_count,video
count,250009.0,250009.000000,250009.000000,250009.000000,250009.000000
mean,530.0,0.943342,3.338908,11.422329,0.373875
std,0.0,21.743664,92.498856,180.956074,0.483832
min,530.0,0.000000,0.000000,0.000000,0.000000
25%,530.0,0.000000,0.000000,0.000000,0.000000
50%,530.0,0.000000,0.000000,1.000000,0.000000
75%,530.0,0.000000,1.000000,3.000000,1.000000
max,530.0,4365.000000,28085.000000,27605.000000,1.000000


In [ ]:
#

In [ ]:
#

In [ ]:
# Create the Bitcoin sentiment scores DataFrame

bitcoin_sentiments = []

for Bitcoin in Bitcoin_tweets["Bitcoin_tweets.csv"]:
    try:
        text = tweets["content"]
        date = tweets["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        bitcoin_sentiments.append({
            "Text": text,
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
btc_df = pd.DataFrame(bitcoin_sentiments)

# Reorder DataFrame columns
cols = ["Compound",  "Negative", "Neutral","Positive","Text"]
btc_df = btc_df[cols]

btc_df.head()

In [ ]:
# Define the search term and the date_since date as variables
search_words = "#bitcoin"
date_since = "2020-01-01"

In [ ]:
# 3000 Tweets about Bitcoin
search_term = '#bitcoin -filter:retweets'
# Create a cursor object
tweets = tweepy.Cursor(api.search, q=search_term, lang='en', since = '2020-01-01', tweet_mode = 'extended'). items(3000)
# Store tweets in a variable and get full text
all_tweets = [tweet.full.text for tweet in tweets]

In [ ]:
# Create a twitterapi client
twitterapi = TwitterApiClient(api_key=api_key)

In [ ]:
# Fetch the Bitcoin news articles
# YOUR CODE HERE!
bitcoin_articles = tweepy.get_everything(
    q="bitcoin",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

In [ ]:
# Fetch the Bitcoin news articles
bitcoin_articles=api_key.getenv(q='bitcoin', language='en', sort_by='relevancy')

In [ ]:
#Sentiment Analysis Bitcoin

def percentage(part,whole):
 return 100 * float(part)/float(whole)
keyword = input("Bitcoin")
noOfTweet = int(input ("3000"))
tweets = tweepy.Cursor(api.search, q=keyword).items(noOfTweet)
positive = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []
for tweet in tweets:
 
 #print(tweet.text)
 tweet_list.append(tweet.text)
 analysis = TextBlob(tweet.text)
 score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
 neg = score['neg']
 neu = score['neu']
 pos = score['pos']
 comp = score['compound']
 polarity += analysis.sentiment.polarity
 
if neg > pos:
     negative_list.append(tweet.text)
     negative += 1
elif pos > neg:
     positive_list.append(tweet.text)
     positive += 1
 
elif pos == neg:
     neutral_list.append(tweet.text)
     neutral += 1
     positive = percentage(positive, noOfTweet)
     negative = percentage(negative, noOfTweet)
     neutral = percentage(neutral, noOfTweet)
     polarity = percentage(polarity, noOfTweet)
     positive = format(positive, '.1f')
     negative = format(negative, '.1f')
     neutral = format(neutral, '.1f')


In [ ]:
# Fetch the Bitcoin news articles
bitcoin_articles=api_key.get_everything(q='bitcoin', language='en', sort_by='relevancy')# Fetch the Ethereum news articles
ethereum_articles=Twitter_API.get_everything(q='ethereum', language='en', sort_by='relevancy')

In [ ]:
# Articles count
ba=bitcoin_articles['totalResults']
ea=ethereum_articles['totalResults']
print(f'There are {ba} articles on Bitcoin and {ea} articles on Ethereum')

In [ ]:
# Create the Bitcoin sentiment scores DataFrame
# YOUR CODE HERE!

In [ ]:
# Create the Ethereum sentiment scores DataFrame
# YOUR CODE HERE!

In [ ]:
# Describe the Bitcoin Sentiment
# YOUR CODE HERE!

In [ ]:
# Describe the Ethereum Sentiment
# YOUR CODE HERE!

### Questions:

Q: Which coin had the highest mean positive score?

A: 

Q: Which coin had the highest compound score?

A: 

Q. Which coin had the highest positive score?

A: 

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [ ]:
# Instantiate the lemmatizer
# YOUR CODE HERE!

# Create a list of stopwords
# YOUR CODE HERE!

# Expand the default stopwords list if necessary
# YOUR CODE HERE!

In [ ]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text

   
    # Create a tokenized list of the words
    
    
    # Lemmatize words into root words

   
    # Convert the words to lowercase
    
    
    # Remove the stop words
    
    
    return tokens

In [ ]:
# Create a new tokens column for Bitcoin
# YOUR CODE HERE!

In [ ]:
# Create a new tokens column for Ethereum
# YOUR CODE HERE!

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [ ]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [ ]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [ ]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [ ]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [ ]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
# YOUR CODE HERE!

In [ ]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [ ]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
# YOUR CODE HERE!

In [ ]:
# List all Entities
# YOUR CODE HERE!

---